<a href="https://colab.research.google.com/github/swiminthewind/colab/blob/main/231121-%E6%B5%8B%E8%AF%95python%E7%89%88%E4%B8%80%E9%98%B6%E8%B0%90%E6%B3%A2%E7%9A%84%E6%AD%A3%E7%A1%AE%E6%80%A7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import ee
ee.Authenticate()
ee.Initialize()

In [13]:
import pandas as pd
import math
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

In [14]:
roi = ee.FeatureCollection("projects/ee-grn/assets/cd_plain_cd")
csPlus= ee.ImageCollection("GOOGLE/CLOUD_SCORE_PLUS/V1/S2_HARMONIZED")
s2sr = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")

In [15]:
timeField = 'system:time_start'
QA_BAND = 'cs'
CLEAR_THRESHOLD = 0.6

def cloud_remove(img):
    return img.updateMask(img.select(QA_BAND).gte(CLEAR_THRESHOLD)).select('.*').copyProperties(img, ['system:time_start'])

composite = s2sr.filterBounds(roi).filterDate('2022-01-01','2023-11-21').linkCollection(csPlus,[QA_BAND]).map(cloud_remove)

def s2addVariables(image):
  date = ee.Date(image.get(timeField))
  years = date.difference(ee.Date('1970-01-01'), 'year')
  return image.addBands(image.normalizedDifference(['B8', 'B4']).rename('NDVI')).addBands(ee.Image(years).rename('t')).float().addBands(ee.Image.constant(1))

filteredSentinel = composite.map(s2addVariables)
print(filteredSentinel.size().getInfo())
print(filteredSentinel.aggregate_array('system:index').getInfo())

1245
['20220101T034141_20220101T034307_T48RUU', '20220101T034141_20220101T034307_T48RUV', '20220101T034141_20220101T034307_T48RVU', '20220101T034141_20220101T034307_T48RVV', '20220104T035141_20220104T035418_T48RTV', '20220104T035141_20220104T035418_T48RUU', '20220104T035141_20220104T035418_T48RUV', '20220104T035141_20220104T035418_T48RVU', '20220104T035141_20220104T035418_T48RVV', '20220106T034129_20220106T034832_T48RUU', '20220106T034129_20220106T034832_T48RUV', '20220106T034129_20220106T034832_T48RVU', '20220106T034129_20220106T034832_T48RVV', '20220109T035129_20220109T035904_T48RTV', '20220109T035129_20220109T035904_T48RUU', '20220109T035129_20220109T035904_T48RUV', '20220109T035129_20220109T035904_T48RVU', '20220109T035129_20220109T035904_T48RVV', '20220111T034121_20220111T034122_T48RUU', '20220111T034121_20220111T034122_T48RUV', '20220111T034121_20220111T034122_T48RVU', '20220111T034121_20220111T034122_T48RVV', '20220114T035121_20220114T035359_T48RTV', '20220114T035121_20220114T03

In [16]:
dataset = filteredSentinel.select(['NDVI','t','constant'])
independents = ee.List(['constant', 't'])
dependent = ee.String('NDVI')

In [17]:
# 一阶谐波
harmonicIndependents_1 = ee.List(['constant', 't', 'cos1', 'sin1'])

def add_sincos_1(image):
  timeRadians1 = image.select('t').multiply(2 * math.pi)
  return image.addBands(timeRadians1.cos().rename('cos1')).addBands(timeRadians1.sin().rename('sin1'))

harmonicSentinel_1 = dataset.map(add_sincos_1)

harmonicTrend_1 = harmonicSentinel_1.select(harmonicIndependents_1.add(dependent)).reduce(ee.Reducer.linearRegression(numX=harmonicIndependents_1.length(),numY=1))

harmonicTrendCoefficients_1 = harmonicTrend_1.select('coefficients').arrayProject([0]).arrayFlatten([harmonicIndependents_1])

def fit_1(image):
  return image.addBands(image.select(harmonicIndependents_1).multiply(harmonicTrendCoefficients_1).reduce('sum').rename('fitted'))

fittedHarmonic_1 = harmonicSentinel_1.map(fit_1)

In [18]:
point = ee.Geometry.Point(103.69442853431933, 30.551562081391808)

rTable = ee.FeatureCollection(fittedHarmonic_1.filterBounds(point).map(lambda image: ee.Feature(
    None,
    {
        'date': image.date().format('yyyy-MM-dd'),
        'NDVI': image.reduceRegion(ee.Reducer.mean(), point, 10).get("NDVI"),
        'fitted': image.reduceRegion(ee.Reducer.mean(), point, 10).get("fitted")
    }
)))

rTable_dict = rTable.getInfo()

dates = [feat['properties']['date'] for feat in rTable_dict['features']]
ndvi_values = [feat['properties']['NDVI'] for feat in rTable_dict['features']]
fitted_values = [feat['properties']['fitted'] for feat in rTable_dict['features']]

df = pd.DataFrame({'Date': dates, 'NDVI': ndvi_values, 'Fitted': fitted_values})

df = df.dropna()

print(df)

           Date      NDVI    Fitted
0    2022-01-01  0.480345  0.575376
11   2022-01-29  0.638256  0.549029
17   2022-02-13  0.727610  0.541487
25   2022-03-05  0.589392  0.539443
26   2022-03-07  0.687620  0.539744
27   2022-03-10  0.715548  0.540369
29   2022-03-15  0.706660  0.541852
30   2022-03-17  0.713792  0.542595
39   2022-04-09  0.631566  0.556933
43   2022-04-19  0.521598  0.566016
44   2022-04-21  0.573421  0.567993
46   2022-04-26  0.480552  0.573165
49   2022-05-04  0.220395  0.581987
51   2022-05-09  0.178203  0.587760
54   2022-05-16  0.197687  0.596071
71   2022-06-28  0.618742  0.645463
74   2022-07-05  0.619651  0.651947
75   2022-07-08  0.668581  0.654510
81   2022-07-23  0.735616  0.664954
82   2022-07-25  0.686726  0.666019
83   2022-07-28  0.771458  0.667474
87   2022-08-07  0.763696  0.670929
88   2022-08-09  0.751144  0.671355
89   2022-08-12  0.808487  0.671827
93   2022-08-22  0.608793  0.671907
99   2022-09-06  0.694187  0.667689
117  2022-10-21  0.278708  0